** Описание **
Построить графики распределения в спам и не спам множествах следующих признаков:

•	Количество слов на странице
•	Количество слов в заголовке страниц (слова в теге <html><head><title > Some text </title>)
•	Средняя длинна слова
•	Количество слов в анкорах ссылок (<html><body><a> Some text </a>) # Общее кол-во или среднее кол-во?



In [ ]:
# !pip install chardet

In [7]:
import sys
import base64
import random
import chardet
import codecs
import re
from re import sub

from nltk.stem import SnowballStemmer
from HTMLParser import HTMLParser

from sklearn.grid_search import GridSearchCV


stemmer = SnowballStemmer("russian");

In [125]:
def search_params(estimator, params, x_train, y_train, scoring="roc_auc", cv=5):
    gsc = GridSearchCV(estimator, params, cv=cv, scoring=scoring)
    gsc.fit(x_train, y_train)
    print("Best parameters set found on development set:")
    print()
    print(gsc.best_params_)
    print()
    print("Grid scores on development set:")
    print()
    for params, mean_score, scores in gsc.grid_scores_:
        print("%0.3f (+/-%0.03f) for %r"
              % (mean_score, scores.std() * 2, params))
    print()
    return gsc.best_estimator_, gsc.best_params_ 

 ** HTML Parser **
Тут мы можем распарсить страницу, выделить текст, заголовок, текст в ссылках документа

In [126]:
from collections import defaultdict

In [127]:
class SpamHTMLParser(HTMLParser):
    def __init__(self):
        HTMLParser.__init__(self)
        self.__text = []
        
        self.data = defaultdict(list)
        self.curr_tag = None

    def handle_starttag(self, tag, attrs):
        if tag == 'img':
            d = dict(attrs)
            if 'alt' in d.keys():
                if d.get('alt'):
                    self.data[self.curr_tag].append(d.get('alt'))
        
        if tag == 'meta':
            if attrs:
                for k, v in attrs:
                    if v:
                        self.data[self.curr_tag].append(v)
        if tag == None:
            print "ahtung!"
        self.curr_tag = tag
            
    def handle_data(self, data):
        if data:
            self.data[self.curr_tag].append(data)
        
        
#         if self.curr_tag == 'meta':
#             return
        
#         if self.curr_tag == 'iframe':
#             print data
            
#         if self.curr_tag:
#             self.data[self.curr_tag].append(data)

#         text = data.strip()
#         if len(text) > 0:
#             text = sub('[ \t\r\n]+', ' ', text)
#             self.__text.append(text + ' ')

            
    def handle_endtag(self, tag):
        self.curr_tag = None

    def text(self):
        return ''.join(self.__text).strip()

** Сбор статистики **
Класс отвечает за сбор статистики

In [3]:
import numpy as np
import codecs

from nltk.corpus import stopwords

In [129]:


class Stat:
    def __init__(self):
            self.points = []
            self.spam = []
            self.notspam = []
                            
def extract_russian(arr):
    if len(arr) == 0: return []
    text = " ".join(arr)
    russian_words = re.findall(u"[\u0400-\u0500]+", text.lower())
    return [w for w in russian_words if w not in stopwords.words('russian')] 
                
class StatsCollector:
    def __init__(self):
        self.total_words = Stat();
        self.header_words = Stat();
        self.average_word_length = Stat();
        self.links_words = Stat();
        
    def collect(self, mark, pageInb64, url):
        html = base64.b64decode(pageInb64).decode('utf-8')
        
        parser = SpamHTMLParser()
        parser.feed(html)
        
    
        with codecs.open("tmp.html", "w", "utf-8") as fout:
            fout.write(html)

        features = defaultdict(dict)
        
        for key, val in parser.data.items():
            try:
                words = extract_russian(val)
                if len(words) != 0:
                    features[key]["words"] = words
                    features[key]["counts"] = len(val)
                    features[key]["mean"] = sum([len(body.split()) for body in val])/float(len(val))
            except TypeError:
                print key, val
                raise TypeError

#                 print key, len(words), len(val), " | ",
#             if key == "br":
#                 for w in words:
#                     print w
        
            
#         print "\n\n"
#         total_words = int(len(parser.text().split(" ")))
#         if "title" in parser.data.keys():
#             header_words = int(np.sum(parser.data["title"]))
#         else:
#             header_words = 0
#         average_word_length = int(np.mean([len(w) for w in parser.text().split(" ")]))
#         if "a" in parser.data.keys():
#             links_words = int(np.sum(parser.data["a"]))
#         else:
#             links_words = 0
    
#         if mark == 0: # notspam
#             self.total_words.notspam.append(total_words)
#             self.header_words.notspam.append(header_words)
#             self.average_word_length.notspam.append(average_word_length)
#             self.links_words.notspam.append(links_words)
            
#         if mark == 1: # notspam
#             self.total_words.spam.append(total_words)
#             self.header_words.spam.append(header_words)
#             self.average_word_length.spam.append(average_word_length)
#             self.links_words.spam.append(links_words)
        
#         self.total_words.points.append(mark)
#         self.header_words.points.append(mark)
#         self.average_word_length.points.append(mark)
#         self.links_words.points.append(mark)
        
        parser.close();
        
        return features
                


** Обрабатываем входной файл **
Входной файл  ./data/train-set-ru-b64-utf-8.txt 
Формат - поля разделенные табуляциями
0 - идентификатор документа
1 - метка класса 0 - не спам, 1 - спам
2 - урл документа
3 - документ в кодировке base64


In [6]:
from collections import Counter

In [ ]:
data = defaultdict(dict)

In [ ]:
len(data.keys())

In [218]:
DATA_FILE  = './data/train-set-ru-b64-utf-8.txt'

stats_collector = StatsCollector()
i_from=0

from urlparse import urlparse
from itertools import chain

with open("urls.txt", 'w') as furls:
    with open (DATA_FILE) as df:
        for i, line in enumerate(df):
            if i < i_from:
                continue

            line = line.strip()
            parts = line.split()
#             print int(parts[1]), parts[2]
            url_o = urlparse(parts[2])
            url_tokens = re.findall(r"[a-z]+", url_o.netloc) + re.findall(r"[a-z]+", url_o.path)
            url_tokens = [token for token in url_tokens if token not in ["ru", "html", "com"]]
            url_tokens = filter(lambda x: len(x) > 3, url_tokens)
    #         tri_gramms = [[''.join(b) for b in zip(w, w[1:], w[2:], w[3:])] for w in url_tokens]
            data[parts[0]]["url_tokens"] = url_tokens

    #         print url_tokens

    #         features = stats_collector.collect(int(parts[1]), parts[3], parts[2])

    #         data[parts[0]]["target"] = int(parts[1])
    #         data[parts[0]]["url"] = parts[2]
    #         data[parts[0]]["features"] = features

    #         sys.stdout.write('\r' + "%s" % (i))
    #         sys.stdout.flush()

    #         
            furls.write("%d %s\n" % (int(parts[1]), parts[2]))
#             if i >= 200:
#                 break

In [217]:
" ".join([None])

TypeError: sequence item 0: expected string, NoneType found

In [7]:
import json

In [8]:

# with open('data.json', 'w') as outfile:
#     json.dump(data, outfile)

In [9]:
with open('data.json') as data_file:    
    data = json.load(data_file)

In [10]:
targets = [v["target"] for k, v in data.iteritems()]

In [11]:
tags = []

for item in data.items():
    tags += item[1]["features"].keys()

In [12]:
from collections import Counter

In [240]:
feature_tags = [None, "p", "div", "a", "title", "h1","h2","h3","h4","h5","h6",
                "span", "strong", "em", "img", "b", "meta",
                "i", "small", "head", "blockquote",
               "cite", "big"]

In [241]:
keys, _ = zip(*filter(lambda v: v[1] > 0, Counter(tags).most_common()))

In [242]:
keys

(u'a',
 u'title',
 u'null',
 u'div',
 u'b',
 u'p',
 u'h1',
 u'span',
 u'strong',
 u'script',
 u'td',
 u'br',
 u'h2',
 u'meta',
 u'h3',
 u'i',
 u'font',
 u'em',
 u'label',
 u'li',
 u'img',
 u'small',
 u'h4',
 u'option',
 u'th',
 u'u',
 u'h5',
 u'center',
 u'input',
 u'nobr',
 u'ins',
 u'dfn',
 u'dt',
 u'link',
 u'blockquote',
 u'form',
 u'h6',
 u'cite',
 u'button',
 u'noindex',
 u'big',
 u'head',
 u'style',
 u'base',
 u'dd',
 u'textarea',
 u'noscript',
 u'legend',
 u'pre',
 u'abbr',
 u'caption',
 u'hr',
 u'index',
 u'ul',
 u'marquee',
 u'left',
 u'address',
 u'article',
 u'sup',
 u's',
 u'blink',
 u'quote',
 u'st1:metricconverter',
 u'sub',
 u'idxskip',
 u'code',
 u'iframe',
 u'wbr',
 u'acronym',
 u'tt',
 u'body',
 u'param',
 u'section',
 u'yabarhighlightertag',
 u'fieldset',
 u'del',
 u'tdclass="dadeno"',
 u'cnter',
 u'tr',
 u'tdid="ubabut"',
 u'norb',
 u'dl',
 u'string',
 u'tdclass="adapusi"',
 u'tdid="adotuno"',
 u'right',
 u'tdclass="lofib"',
 u'odg',
 u'basefont',
 u'bb',
 u'stong'

# Только на числовых

In [243]:
# кол-во слов
# кол-во каждого из тегов > 65
# общее кол-во тегов < 65
# среднее кро-во слов в теге

In [244]:
import numpy as np

In [245]:
numeric_data = []

for id, item in data.items():
    obj = []
    obj.append(item["target"])
    
    for key in set(feature_tags):# - set([None, "p", "div"]):
        if key in item["features"].keys():
            obj.append(item["features"][key]["counts"])
            obj.append(item["features"][key]["mean"])
            obj.append(len(item["features"][key]["words"]))
            obj.append(len(Counter(item["features"][key]["words"]).keys()))
        else:
            obj.append(0)
            obj.append(0)
            obj.append(0)
            obj.append(0)
            
    numeric_data.append(obj)

In [246]:
np.unique([len(a) for a in numeric_data])

array([93])

In [247]:
numeric_data = np.array(numeric_data, dtype=np.float)

In [248]:
data1 = numeric_data

In [249]:
from sklearn.cross_validation import train_test_split

In [250]:
X_train, X_test, y_train, y_test = train_test_split(data1[:, 1:], data1[:, 0], test_size=0.33, random_state=42)

In [251]:
from sklearn.linear_model import LogisticRegression

In [252]:
lg = LogisticRegression(penalty="l1", C=1)

In [253]:
lg.fit(X_train, y_train)

LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr',
          penalty='l1', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0)

In [254]:
y_pred = lg.predict(X_test)

In [255]:
from sklearn.metrics import f1_score

In [256]:
f1_score(y_test, y_pred)

0.8756679729248309

In [257]:
from sklearn.calibration import CalibratedClassifierCV
ccc = CalibratedClassifierCV(lg)
ccc.fit(X_train, y_train)

CalibratedClassifierCV(base_estimator=LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr',
          penalty='l1', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0),
            cv=3, method='sigmoid')

In [258]:
f1_score(y_test, ccc.predict(X_test))

0.87030138841855742

In [259]:
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
gb = GradientBoostingClassifier(n_estimators=400)

In [260]:
gb.fit(X_train, y_train)

GradientBoostingClassifier(init=None, learning_rate=0.1, loss='deviance',
              max_depth=3, max_features=None, max_leaf_nodes=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=400,
              random_state=None, subsample=1.0, verbose=0,
              warm_start=False)

In [261]:
y_pred = gb.predict(X_test)
f1_score(y_test, y_pred)

0.92149669845928106

In [262]:
rfc = RandomForestClassifier(n_estimators=500, n_jobs=-1)

In [263]:
rfc.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=500, n_jobs=-1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [264]:
y_pred = rfc.predict(X_test)
f1_score(y_test, y_pred)

0.93868450390189517

# С текстом

In [265]:
feature_tags

[None,
 'p',
 'div',
 'a',
 'title',
 'h1',
 'h2',
 'h3',
 'h4',
 'h5',
 'h6',
 'span',
 'strong',
 'em',
 'img',
 'b',
 'meta',
 'i',
 'small',
 'head',
 'blockquote',
 'cite',
 'big']

In [293]:
text_data = []

for id, item in data.items():
    bag = []
    
    for key in keys:
        if key in item["features"].keys():
            words = item["features"][key]["words"]
            bag += words
            bag += [''.join(b) for b in zip(words, words[1:])]
            bag += [''.join(b) for b in zip(words, words[1:], words[2:])]
        
#     bag += item["url_tokens"]

    text_data.append(" ".join(bag))

In [294]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [295]:
tfidf = TfidfVectorizer(use_idf=True, ngram_range=(1, 1), sublinear_tf=True)

In [296]:
data2 = tfidf.fit_transform(text_data)

In [334]:
data2.shape

(7044, 10387671)

In [328]:
X_train, X_test, y_train, y_test = train_test_split(data2, data1[:, 0], test_size=0.33, random_state=42)

In [329]:
y_train[y_train == 0] = -1
y_test[y_test == 0] = -1

In [330]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.metrics import f1_score

In [331]:
lg = LogisticRegression(penalty="l1", C=100)

In [332]:
lg.fit(X_train, y_train)

LogisticRegression(C=100, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr',
          penalty='l1', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0)

In [333]:
y_pred = lg.predict(X_test)
f1_score(y_test, y_pred)

0.96098104793756967

In [315]:
from sklearn.externals import joblib

In [304]:
from sklearn.calibration import CalibratedClassifierCV
ccc = CalibratedClassifierCV(lg)
ccc.fit(X_train, y_train)

CalibratedClassifierCV(base_estimator=LogisticRegression(C=100, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr',
          penalty='l1', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0),
            cv=3, method='sigmoid')

In [316]:
!mkdir models

In [317]:
joblib.dump(ccc, './models/model.pkl')

['./models/model.pkl',
 './models/model.pkl_01.npy',
 './models/model.pkl_02.npy',
 './models/model.pkl_03.npy',
 './models/model.pkl_04.npy',
 './models/model.pkl_05.npy',
 './models/model.pkl_06.npy',
 './models/model.pkl_07.npy',
 './models/model.pkl_08.npy',
 './models/model.pkl_09.npy',
 './models/model.pkl_10.npy',
 './models/model.pkl_11.npy',
 './models/model.pkl_12.npy',
 './models/model.pkl_13.npy',
 './models/model.pkl_14.npy',
 './models/model.pkl_15.npy',
 './models/model.pkl_16.npy']

In [305]:
y_pred = ccc.predict(X_test)
f1_score(y_test, y_pred)

0.96399099774943731

In [306]:
lsvc = LinearSVC()

In [307]:
lsvc.fit(X_train, y_train)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [308]:
y_pred = lsvc.predict(X_test)
f1_score(y_test, y_pred)

0.95868998883513223

In [309]:
from sklearn.naive_bayes import MultinomialNB

In [310]:
mnb = MultinomialNB(alpha=0.4)
mnb.fit(X_train, y_train)

MultinomialNB(alpha=0.4, class_prior=None, fit_prior=True)

In [311]:
y_pred = mnb.predict(X_test)
f1_score(y_test, y_pred)

0.92717584369449391

In [312]:
from sklearn.cross_validation import cross_val_score

In [313]:
cvs = cross_val_score(lg, data2, data1[:, 0], scoring="f1", cv=5, n_jobs=-8)

In [314]:
cvs

array([ 0.96323086,  0.9599013 ,  0.96656535,  0.95555556,  0.95609153])

# Text + Numeric

In [112]:
from scipy.sparse import coo_matrix, hstack

In [113]:
coo_matrix(data1[:, 1:]).shape, data2.shape

((7044, 366), (7044, 4352812))

In [114]:
data3 = hstack([coo_matrix(data1[:, 1:]), data2])

In [115]:
data3.shape

(7044, 4353178)

In [116]:
X_train, X_test, y_train, y_test = train_test_split(data3, data1[:, 0], test_size=0.33, random_state=42)

In [117]:
y_train[y_train == 0] = -1
y_test[y_test == 0] = -1

In [122]:
lsvc = LinearSVC(C=0.01)
lsvc.fit(X_train, y_train)

LinearSVC(C=0.01, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [123]:
y_pred = lsvc.predict(X_test)
f1_score(y_test, y_pred)

0.89123071132856602

In [ ]:
lg = LogisticRegression(penalty="l1", C=100)
lg.fit(X_train, y_train)

In [ ]:
y_pred = lg.predict(X_test)
f1_score(y_test, y_pred)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=400, n_jobs=-1)

In [ ]:
rf.fit(X_train, y_train)

In [ ]:
y_pred = rf.predict(X_test)
f1_score(y_test, y_pred)

# Vowpal Wabbit

In [ ]:
with codecs.open("data.txt", 'w', "utf-8") as fout:
    for id, item in data.items():
        bag = []
        target = -1 if item["target"] == 0 else 1

        for key in feature_tags:
            if key in item["features"].keys():
                bag += [w for w in item["features"][key]["words"] if len(w) < 10]


        fout.write(u'{0} | {1}\n'.format(target, ' '.join(bag)))

In [158]:
!head -n 2 data.txt

-1 | яндекс круг поиск почта календарь деньги круг фотки народ открытки авто афиша блоги видео картинки карты каталог маркет музыка новости погода работа словари услуги ру программы сервисы войти помощь главная работа компании услуги вспомнить пароль создать профиль фото фото мобильная версия помощь обратная связь блог яндекс колл москва круг люди круге колл москва ещё м россия москва область москва е россия москва область москва
-1 | яндекс круг поиск почта календарь деньги круг фотки народ открытки авто афиша блоги видео картинки карты каталог маркет музыка новости погода работа словари услуги ру программы сервисы войти помощь главная работа компании услуги вспомнить пароль создать профиль фото фото мобильная версия помощь обратная связь блог яндекс люди которые сайт круг люди круге ещё елена россия татарстан казань л россия петербург область санкт петербург например


In [8]:
# !vw --loss_function "logistic" -b 31 -d data.txt -c -k -f model.vw --passes 10 -l 0.1 